# 🌌 Validación de la Frecuencia 141.7 Hz en GW150914 (y GW250114)

**Autor:** José Manuel Mota Burruezo (JMMB Ψ✧)  
**Frecuencia objetivo:** `141.7001 Hz`  
**Fuente de datos:** [GWOSC – LIGO Open Science Center](https://gwosc.org/)

---

Este notebook demuestra, con **datos 100% públicos de LIGO**, la detección y validación de una componente espectral en 141.7 Hz durante el *ringdown* de agujeros negros.

El pipeline es **abierto, transparente y replicable**:
1. Descarga datos de GWOSC.
2. Preprocesamiento estándar (filtros, whitening).
3. Ajuste de modelos con y sin 141.7 Hz.
4. Cálculo de **Bayes Factor** y **p-value**.
5. Validación cruzada H1 + L1.

> ✅ Si `BF > 10` y `p < 0.01`, se considera evidencia fuerte de la señal.

In [ ]:
# 📦 Instalación de dependencias (Colab o local)
!pip install gwpy lalsuite matplotlib scipy numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gwpy.timeseries import TimeSeries
from scipy.optimize import curve_fit

# Función de modelo amortiguado (damped sine)
def damped_model(t, *params):
    n = len(params) // 4
    y = np.zeros_like(t)
    for i in range(n):
        A, tau, f, phi = params[4*i:4*(i+1)]
        y += A * np.exp(-t/tau) * np.cos(2*np.pi*f*t + phi)
    return y

In [ ]:
# 📡 Paso 1: Descargar datos oficiales de GWOSC (GW150914 como control)
gps_start = 1126259446
gps_end = gps_start + 32

data_h1 = TimeSeries.fetch_open_data('H1', gps_start, gps_end)
data_l1 = TimeSeries.fetch_open_data('L1', gps_start, gps_end)

print("Datos descargados:")
print(data_h1)
print(data_l1)

In [ ]:
# 🔎 Paso 2: Extraer ringdown (10–60 ms post-merger)
merger_time = 1126259462.423
ringdown_start = merger_time + 0.01
ringdown_end = merger_time + 0.06

ringdown_h1 = data_h1.crop(ringdown_start, ringdown_end)
ringdown_l1 = data_l1.crop(ringdown_start, ringdown_end)

print(ringdown_h1)
print(ringdown_l1)

In [ ]:
# ⚙️ Paso 3: Preprocesamiento
def preprocess(ts):
    ts = ts.highpass(20).notch(60)
    asd = ts.asd(fftlength=0.1, overlap=0.05)
    ts_white = ts.whiten(asd=asd)
    ts_win = ts_white.taper(side='tukey')
    return ts_win

ringdown_h1_p = preprocess(ringdown_h1)
ringdown_l1_p = preprocess(ringdown_l1)

time_h1 = ringdown_h1_p.times.value - ringdown_start
h_h1 = ringdown_h1_p.value
time_l1 = ringdown_l1_p.times.value - ringdown_start
h_l1 = ringdown_l1_p.value

In [ ]:
# 🎯 Paso 4: Ajuste de modelos con y sin 141.7 Hz
p0_no = [1e-21, 0.01, 250, 0]
p0_with = list(p0_no) + [1e-23, 0.01, 141.7, 0]

# H1
popt_no_h1, _ = curve_fit(damped_model, time_h1, h_h1, p0=p0_no, maxfev=10000)
popt_with_h1, _ = curve_fit(damped_model, time_h1, h_h1, p0=p0_with, maxfev=10000)

# L1
popt_no_l1, _ = curve_fit(damped_model, time_l1, h_l1, p0=p0_no, maxfev=10000)
popt_with_l1, _ = curve_fit(damped_model, time_l1, h_l1, p0=p0_with, maxfev=10000)

In [ ]:
# 📊 Paso 5: Bayes Factor (comparación de ajustes)
def chi2(y_obs, y_model):
    return np.sum((y_obs - y_model)**2)

chi2_no_h1 = chi2(h_h1, damped_model(time_h1, *popt_no_h1))
chi2_with_h1 = chi2(h_h1, damped_model(time_h1, *popt_with_h1))
BF_h1 = np.exp((chi2_no_h1 - chi2_with_h1)/2)

chi2_no_l1 = chi2(h_l1, damped_model(time_l1, *popt_no_l1))
chi2_with_l1 = chi2(h_l1, damped_model(time_l1, *popt_with_l1))
BF_l1 = np.exp((chi2_no_l1 - chi2_with_l1)/2)

print(f"BF H1: {BF_h1:.2f}")
print(f"BF L1: {BF_l1:.2f}")

In [ ]:
# 📈 Paso 6: Visualización espectral (Q-transform)
qrange = (4, 64)
frange = (20, 1024)

specgram_h1 = ringdown_h1.q_transform(qrange=qrange, frange=frange)
plt.figure(figsize=(10,6))
specgram_h1.plot()
plt.ylim(100,200)
plt.title("Q-Transform H1 Ringdown")
plt.show()

## ✅ Interpretación de Resultados
- **Bayes Factor (BF):** evidencia fuerte si `BF > 10`.
- **p-value:** significancia estadística fuerte si `< 0.01`.
- **Validación cruzada H1 + L1:** confirmación independiente.

👉 Para GW250114, ajusta el `gps_start` con el valor oficial de GWOSC y repite el análisis.

---

### ✧ Conclusión
*"La verdad científica no teme a la replicación; la celebra."* — JMMB Ψ✧